<a href="https://colab.research.google.com/github/ashishagg70/SentimentAnalysisBasicDLNLP/blob/master/CS772_Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !gdown --id 1E3lMHlwk45uQp2Ufq6LprOapHsYaBCYM
!gdown --id 1aVV9n3hIzqqM9u3GRl2bcNDl2knhFPvF

Downloading...
From: https://drive.google.com/uc?id=1aVV9n3hIzqqM9u3GRl2bcNDl2knhFPvF
To: /content/SentimentAnalysisData.zip
2.65MB [00:00, 42.0MB/s]


In [ ]:
# !gunzip cc.en.300.vec.gz
!unzip SentimentAnalysisData.zip

Archive:  SentimentAnalysisData.zip
  inflating: SentimentAnalysisData/stopwords.txt  
  inflating: SentimentAnalysisData/vocab.csv  
  inflating: SentimentAnalysisData/test.csv  
  inflating: SentimentAnalysisData/train.csv  
  inflating: SentimentAnalysisData/gold_test.csv  


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 21.9MB/s 
     |████████████████████████████████| 901kB 59.5MB/s 
     |████████████████████████████████| 3.3MB 55.1MB/s 


In [ ]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
from collections import defaultdict
import math
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
import os
from transformers import BertTokenizer, TFBertModel


nltk.download('punkt')

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="grpc://" + os.environ['COLAB_TPU_ADDR'])
# #Connect to the TPU handle and initialise it
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver)

KeyError: ignored

In [ ]:
tf.random.set_seed(1)
np.random.seed(1)
np.set_printoptions(threshold=np.inf)

max_len=50
NUMBER_CLASSES = 5
EMBDIM=300
hidden_layer_count=0
hidden_layer_numunits=[256]
filepath = './SentimentAnalysisData/'
word_to_index = defaultdict(int)
index_to_word = defaultdict(str)
bert_model_name = 'bert-base-uncased'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert = TFBertModel.from_pretrained(bert_model_name)
for layer in bert.layers:
    layer.trainable = False

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
max_sentence_length = 100 #tokenizer.max_model_input_sizes[bert_model_name]
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id

In [ ]:
def encode_data(text):
    text_encoded = np.full((len(text),max_sentence_length), pad_token_idx)
    for i,sent in enumerate(text):
        text_encoded[i][0] = init_token_idx
        tokens = tokenizer.convert_tokens_to_ids(sent)
        l = len(tokens)
        text_encoded[i][1:l+1] = tokens
        text_encoded[i][l+1]=eos_token_idx
    return text_encoded
    

In [ ]:
def remove_stopwords(text):
    newtext = []
    f=open(filepath+'stopwords.txt','r')
    lines=f.readlines()
    stopwordList = [line.rstrip() for line in lines]
    for tokens in text:
        newtext.append([w for w in tokens if not w in stopwordList])
    return newtext

In [ ]:
def perform_tokenization(text):
    return [tokenizer.tokenize(sent) for sent in text]

In [ ]:
def preprocess_data(data, isTrain=True):
    #review = convert_to_lower(review)
    #review = remove_punctuation(review)
    review = perform_tokenization(data)
    #review = remove_stopwords(review)
    review = encode_data(review)    
    return review

In [ ]:
class NeuralNet:

    def __init__(self, reviews, ratings, val_reviews,val_ratings):
        self.reviews = reviews
        self.ratings = tf.keras.utils.to_categorical(y=ratings-1,num_classes=NUMBER_CLASSES)
        val_ratings = tf.keras.utils.to_categorical(y=val_ratings-1,num_classes=NUMBER_CLASSES)
        self.val_data=(val_reviews,val_ratings)
        self.model = None

    def build_nn(self,hidden_layer_count,hidden_layer_numunits):
        #with strategy.scope():
        sentence_indices = tf.keras.layers.Input(shape=(max_sentence_length,),dtype='int32')
        X = bert(sentence_indices)[0]

        X = tf.keras.layers.Flatten()(X)

        # X = tf.keras.layers.SimpleRNN(256,return_sequences=True)(X)

        # X = tf.keras.layers.SimpleRNN(512)(X)
        # X = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256,return_sequences=True,dropout=0.2))(X)
        # X = tf.keras.layers.GRU(256)(X)
    
        for i in range(hidden_layer_count):
            X = tf.keras.layers.Dense(units=hidden_layer_numunits[i],activation = 'relu')(X)
            X = tf.keras.layers.Dropout(rate=0.2)(X)
            # X=tf.keras.layers.Activation(activation=activation_name)(X)
        
        X = tf.keras.layers.Dense(units=5)(X)

        X = tf.keras.activations.softmax(X,axis =1)
        
        self.model = tf.keras.Model(inputs=sentence_indices,outputs=X)
        self.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
        self.model.summary()

    def train_nn(self, batch_size, epochs, balanced):
        
        es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=1,patience=3)
        #self.model.fit(Generator(self.reviews,self.ratings,batch_size), epochs=epochs, batch_size=batch_size,validation_data=self.val_data)
        # else:
        self.model.fit(self.reviews,self.ratings, epochs=epochs, batch_size=batch_size,validation_data=self.val_data)
   
    def predict(self, reviews):
        reviews = np.array(reviews, dtype='float32')
        return np.argmax(self.model.predict(reviews), axis=1) + 1
    
    def predictWithPr(self, reviews):
        reviews = np.array(reviews, dtype='float32')
        return self.model.predict(reviews)

In [ ]:
class Generator(tf.keras.utils.Sequence):
    # Class is a dataset wrapper for better training performance
    def __init__(self, x_set, y_set, batch_size=256):
        self.x, self.y = x_set, y_set
        y_temp=np.argmax(y_set,axis=1)+1
        self.batch_size = batch_size
        
        print(x_set.shape,y_set.shape)
            
    
        self.c1_review = x_set[y_temp==1,:]
        self.c2_review = x_set[y_temp==2,:]
        self.c3_review = x_set[y_temp==3,:]
        self.c4_review = x_set[y_temp==4,:]
        self.c5_review = x_set[y_temp==5,:]
        
        
        
        self.len1=len(self.c1_review)
        self.len2=len(self.c2_review)
        self.len3=len(self.c3_review)
        self.len4=len(self.c4_review)
        self.len5=len(self.c5_review)

        self.c1_ratings = y_set[y_temp==1,:]
        self.c2_ratings = y_set[y_temp==2,:]
        self.c3_ratings = y_set[y_temp==3,:]
        self.c4_ratings = y_set[y_temp==4,:]
        self.c5_ratings = y_set[y_temp==5,:]
        self.batch_1=int(batch_size*0.199)
        self.batch_2=int(batch_size*0.199)
        self.batch_3=int(batch_size*0.199)
        self.batch_4=int(batch_size*0.199)
        self.batch_5=self.batch_size-self.batch_1-self.batch_2-self.batch_3-self.batch_4

    def __len__(self):
        return math.ceil(self.x.shape[0] / self.batch_size)

    def __getitem__(self, idx):
        batch_x=[]
        batch_y=[]
        r = np.random.rand()
        if r<0.2:
          idx1=np.random.choice(np.arange(self.len1),replace=False,size=1)
          batch_x.extend(self.c1_review[idx1,:])
          batch_y.extend(self.c1_ratings[idx1,:])
        elif r<0.4:
          idx2=np.random.choice(np.arange(self.len2),replace=False,size=1)
          batch_x.extend(self.c2_review[idx2,:])
          batch_y.extend(self.c2_ratings[idx2,:])
        elif r<0.6:
          idx3=np.random.choice(np.arange(self.len3),replace=False,size=1)
          batch_x.extend(self.c3_review[idx3,:])
          batch_y.extend(self.c3_ratings[idx3,:])

        elif r<0.8:
          idx4=np.random.choice(np.arange(self.len4),replace=False,size=1)
          batch_x.extend(self.c4_review[idx4,:])
          batch_y.extend(self.c4_ratings[idx4,:])

        else:
          idx5=np.random.choice(np.arange(self.len5),replace=False,size=1)        
          batch_x.extend(self.c5_review[idx5,:])
          batch_y.extend(self.c5_ratings[idx5,:])


        batch_x=np.array(batch_x)
        
        batch_y=np.array(batch_y)
        
        return batch_x, batch_y
    

In [ ]:
train_data = pd.read_csv(filepath+"train.csv")
test_data = pd.read_csv(filepath+"test.csv")

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_data, test_size=0.2)

In [ ]:
train_ratings = np.array(train_df["ratings"])
train_reviews = np.array(preprocess_data(train_df["reviews"]))
val_ratings = np.array(val_df["ratings"])
val_reviews = np.array(preprocess_data(val_df["reviews"], False))
test_reviews = preprocess_data(test_data["reviews"], False)

In [ ]:
# train_ratings = np.array(train_data["ratings"])
# train_reviews = np.array(preprocess_data(train_data))
# test_reviews = preprocess_data(test_data, False)

In [ ]:
model = NeuralNet(train_reviews, train_ratings, val_reviews,val_ratings)
model.build_nn(0,[256])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
batch_size, epochs = 256, 100

In [ ]:
model.train_nn(batch_size, epochs,balanced=False)

Epoch 1/100
157/157 [==============================] - 373s 2s/step - loss: 5.9750 - accuracy: 0.5613 - val_loss: 1.4933 - val_accuracy: 0.6902
Epoch 2/100
157/157 [==============================] - 359s 2s/step - loss: 1.2455 - accuracy: 0.6690 - val_loss: 0.8444 - val_accuracy: 0.7053
Epoch 3/100
  2/157 [..............................] - ETA: 4:47 - loss: 0.7934 - accuracy: 0.7188

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

print("=================Train data evaluation metrices==========================")
# evaluation_matrices(train_ratings, model.predict(train_reviews))

print(accuracy_score(train_ratings,model.predict(train_reviews)))
print(precision_recall_fscore_support(train_ratings,model.predict(train_reviews),average='weighted'))

=================Train data evaluation metrices==========================


KeyboardInterrupt: ignored

In [ ]:
print("=================Test data evaluation metrices==========================")

testPredictions = model.predict(test_reviews)
test_ground_truth = np.array(pd.read_csv(filepath+'gold_test.csv')['ratings'])

# evaluation_matrices(test_ground_truth, testPredictions)
print(accuracy_score(test_ground_truth, testPredictions))
print(precision_recall_fscore_support(test_ground_truth, testPredictions,average='weighted'))

=================Test data evaluation metrices==========================
0.6736
(0.6371299055478612, 0.6736, 0.6351186688386713, None)


In [ ]:
!rm runs/*.txt

In [ ]:
!./run_training3.sh

rnn_r1_trained_e40_b256_h0
2021-04-07 17:12:16.097569: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-07 17:12:17.430161: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-07 17:12:17.430961: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-07 17:12:17.445762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-07 17:12:17.446367: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-04-07 17:12:17.446420: I tensorflow/s

In [ ]:
!tar -cvzf nlpa3runs3_correct.tar.gz runs

runs/
runs/rnn_r1_trained_e40_b256_h0.txt
runs/bilstm_r1_trained_e40_b256_h2.txt
runs/bigru_r1_trained_e40_b256_h1.txt
runs/rnn_r1_trained_e40_b256_h1.txt
runs/lstm_r1_trained_e40_b256_h0.txt
runs/bigru_r1_trained_e40_b256_h0.txt
runs/bigru_r1_trained_e40_b256_h2.txt
runs/gru_r1_trained_e40_b256_h0.txt
runs/bilstm_r1_trained_e40_b256_h0.txt
runs/rnn_r1_trained_e40_b256_h2.txt
runs/lstm_r1_trained_e40_b256_h2.txt
runs/lstm_r1_trained_e40_b256_h1.txt
runs/.ipynb_checkpoints/
runs/gru_r1_trained_e40_b256_h1.txt
runs/gru_r1_trained_e40_b256_h2.txt
runs/bilstm_r1_trained_e40_b256_h1.txt


In [ ]:
ip_data = pd.read_csv("input.csv")
ip_reviews = preprocess_data(ip_data, False)
pred = model.predictWithPr(ip_reviews)

print(pred)

[[3.8932176e-05 2.4177318e-05 9.3062605e-05 2.2082604e-03 9.9763560e-01]
 [6.3922447e-01 5.1454931e-02 6.8178147e-02 5.2719958e-02 1.8842255e-01]]


In [ ]:
tf.keras.utils.plot_model(
    model.model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=False,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

In [ ]:
ip_data["rating"]=np.argmax(pred, axis=1) + 1
ip_data


,Unnamed: 0,reviews,rating
0,0,Amazing!! I love and swear by this stuff. A mu...,5
1,1,This product came in pieces .... would NOT rec...,1


In [ ]:
np.sum(pred,axis=1)

array([1.       , 1.0000001], dtype=float32)

In [ ]:
model.model.save(filepath='~/CS772/')

INFO:tensorflow:Assets written to: ~/CS772/assets


In [ ]:
without_hidden_layer(word2vec_300d)
=================Train data evaluation metrices==========================
              precision    recall  f1-score   support

           1       0.89      0.92      0.90      3229
           2       0.81      0.94      0.87      1766
           3       0.72      0.86      0.78      2915
           4       0.76      0.55      0.64      5519
           5       0.92      0.94      0.93     26571

    accuracy                           0.88     40000
   macro avg       0.82      0.84      0.83     40000
weighted avg       0.88      0.88      0.87     40000

=================Test data evaluation metrices==========================
              precision    recall  f1-score   support

           1       0.63      0.64      0.64      1271
           2       0.23      0.21      0.22       630
           3       0.31      0.40      0.35       911
           4       0.36      0.22      0.27      1404
           5       0.83      0.88      0.86      5784

    accuracy                           0.67     10000
   macro avg       0.47      0.47      0.47     10000
weighted avg       0.66      0.67      0.66     10000

loss: 0.4769 - accuracy: 0.8615 - val_loss: 0.7254 - val_accuracy: 0.7344

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

In [ ]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from collections import defaultdict
import math
from sklearn.model_selection import train_test_split
import sys
from sklearn.metrics import classification_report,confusion_matrix,precision_recall_fscore_support

max_len=50
NUMBER_CLASSES = 5
EMBDIM=300
hidden_layer_count=0
hidden_layer_numunits=[256]

fastText_file='./cc.en.300.vec'

word_to_index = defaultdict(int)

index_to_word = defaultdict(str)

filepath = './SentimentAnalysisData/'

embeddings={}

def pretrain_embedding():
    global embeddings
    f=open(fastText_file, 'r')
    lines=f.readlines()
    f.close()
    for line in lines[1:]:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings[word] = vector

def train_embedding(data, create_new=False):
    global embeddings
    if create_new == True:
        reviews_mod = []
        for sent in data:
            reviews_mod.append(list(map(lambda x:index_to_word[int(x)],sent)))
        model = gensim.models.Word2Vec(reviews_mod, min_count=1,
                                       size=EMBDIM, window=5)
        model.wv.save("train_word2vec2.wordvectors")
    embeddings = KeyedVectors.load("train_word2vec.wordvectors", mmap='r')

def convert_to_lower(text):
    return [i.lower() for i in text]

def remove_punctuation(text):
    return [i.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))) for i in text]

def encode_data(text,isTrain=True):
    global word_to_index
    global index_to_word
    if isTrain:
        f = open(filepath+'vocab.csv','r')
        lines = f.readlines()
        f.close()
        for line in lines:
            li = list(line.split(","))
            index= li[1].rstrip()
            word_to_index[li[0]] =index
            index_to_word[index] = li[0]
    m=len(text)
    text_encoded = np.zeros((m,max_len))
    for i in range(m):
        sentence_words = text[i]
        j = 0
        for w in sentence_words:
            text_encoded[i, j] = word_to_index.get(w)
            j = j + 1
      
    return text_encoded

def remove_stopwords(text):
    newtext = []
    f=open(filepath+'stopwords.txt','r')
    lines=f.readlines()
    stopwordList = [line.rstrip() for line in lines]
    for tokens in text:
        newtext.append([w for w in tokens if not w in stopwordList])
    return newtext

def perform_tokenization(text):
    return [word_tokenize(i) for i in text]

def preprocess_data(data, isTrain=True):
    review = data["reviews"]
    review = convert_to_lower(review)
    review = remove_punctuation(review)
    review = perform_tokenization(review)
    review = remove_stopwords(review)
    review = encode_data(review,isTrain)
    print(review[0][0],type(review[0][0]))    
    return review

In [ ]:
train_data = pd.read_csv(filepath+"train.csv")
test_data = pd.read_csv(filepath+"test.csv")
train_df, val_df = train_test_split(train_data, test_size=0.2)

train_ratings = np.array(train_df["ratings"])
train_reviews = np.array(preprocess_data(train_df))

val_ratings = np.array(val_df["ratings"])
val_reviews = np.array(preprocess_data(val_df, False))

test_reviews = preprocess_data(test_data, False)


10592.0 <class 'numpy.float64'>
19.0 <class 'numpy.float64'>
765.0 <class 'numpy.float64'>


In [ ]:
train_embedding(train_reviews,False)

In [ ]:
vocab_size = len(word_to_index) + 1           
emb_dim = 300
emb_matrix = np.zeros((vocab_size,emb_dim))
for word, idx in word_to_index.items():
    print(word,type(idx))
    print(embeddings[word])
    emb_matrix[idx, :] = embeddings[word]
    break

lustray <class 'str'>
[ 1.02516040e-02 -8.22799653e-03  5.82422689e-03 -9.90779046e-03
 -1.31423725e-02 -1.10141831e-02  4.77552367e-03 -7.84778036e-03
  4.44685295e-03 -3.47391330e-02  3.22807878e-02 -1.71673410e-02
  1.60132721e-03 -1.18543720e-02  1.36441216e-02  6.08943542e-03
 -1.01423450e-02  4.45494102e-03  1.51620582e-02 -1.01185525e-02
 -1.51702706e-02 -5.36216004e-03  7.99301546e-03  3.69705050e-03
 -1.12853190e-02 -9.87657905e-03  3.67454626e-03  4.27600835e-03
  4.71022772e-03  1.61849987e-02 -6.56094961e-03  4.79328958e-03
  7.95631669e-03  2.63434998e-03 -5.81754884e-03  1.75653072e-03
  4.00712295e-03 -2.40791198e-02  7.65394885e-03  2.93672993e-03
  4.48172307e-03  2.40508020e-02 -9.16617643e-03 -5.35608968e-03
 -5.00058616e-03  7.23482436e-03 -2.19398248e-03  1.02893915e-02
  3.24336141e-02  2.10797396e-02  1.05699040e-02  1.68616772e-02
 -1.04280224e-03 -1.08628976e-03 -9.91606619e-03  1.79678816e-02
 -3.67323472e-03  3.43719800e-03  2.71764360e-02 -4.51593613e-03
 -1

IndexError: ignored

In [ ]:
np.array(word_to_index.get('lustray'))

array('1', dtype='<U1')